!python --version

In [ ]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install      datasets==1.13.3
!pip install      transformers==4.11.3
!pip install      librosa
!pip install      jiwer
!pip install      s3fs

In [2]:
%%capture 
!pip install boto3

In [23]:
import boto3
import datasets
import s3fs
import os


In [12]:
# my_session = boto3.session.Session()
# s3 = my_session.client('s3')

In [3]:
# insert AWS keys here


In [4]:
s3 = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'])


In [25]:
# %rm -rf sloveni

In [39]:
BUCKET_NAME = 'commonvoicesdataset'
key = 'Sloveni/vocab.json'

root = '/home/ec2-user/SageMaker/STT'
lang = 'sloveni'
dataset = 'dataset'
tokenizer = 'tokenizer'
json = 'json'
# add s3 path as well  

In [51]:
#create language folder 
lang_dir = os.path.join(root,lang)
if not os.path.isdir(lang_dir):
    os.mkdir(lang_dir)

dataset_path = os.path.join(root,lang,dataset)
if not os.path.isdir(dataset_path):
    os.mkdir(dataset_path)
tokenizer_path = os.path.join(root,lang,tokenizer)


In [52]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "sl", split="train+validation",cache_dir=dataset_path)
common_voice_test = load_dataset("common_voice", "sl", split="test",cache_dir=dataset_path)

Downloading:   0%|          | 0.00/223M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /home/ec2-user/SageMaker/STT/sloveni/dataset/common_voice/sl/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


Reusing dataset common_voice (/home/ec2-user/SageMaker/STT/sloveni/dataset/common_voice/sl/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [53]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [8]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2594
})

In [54]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [55]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


  0%|          | 0/2594 [00:00<?, ?ex/s]

  0%|          | 0/881 [00:00<?, ?ex/s]

In [56]:
common_voice_test

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 881
})

In [57]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)


  0%|          | 0/2594 [00:00<?, ?ex/s]

  0%|          | 0/881 [00:00<?, ?ex/s]

In [58]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2594
})

In [59]:
%pwd

'/home/ec2-user/SageMaker/STT'

In [61]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [62]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [65]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)


In [66]:
import json

# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)
s3.Object('commonvoicesdataset', 'Sloveni/vocab.json').put(Body=json.dumps(vocab_dict,indent = 3))

{'ResponseMetadata': {'RequestId': 'X6M3GVZ49182J45H',
  'HostId': '5s6HjrbrE0TQTBGRbk644JdW/5uLx1FuCCRm8DiXtPCSRJ3KmDXg8ncso5eVjKyiBN9+trIk/xE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '5s6HjrbrE0TQTBGRbk644JdW/5uLx1FuCCRm8DiXtPCSRJ3KmDXg8ncso5eVjKyiBN9+trIk/xE=',
   'x-amz-request-id': 'X6M3GVZ49182J45H',
   'date': 'Wed, 23 Feb 2022 15:36:24 GMT',
   'etag': '"5349b65e07578a45362e1997e1880ac4"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5349b65e07578a45362e1997e1880ac4"'}

In [68]:
#download from correct language 
!aws s3 cp s3://commonvoicesdataset/Sloveni/vocab.json '/home/ec2-user/SageMaker/STT/'   

download: s3://commonvoicesdataset/Sloveni/vocab.json to ./vocab.json


In [69]:
# this tokenizer is using the previous json file/ so becareful what u load
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [70]:
tokenizer.save_pretrained(tokenizer_path)

('/home/ec2-user/SageMaker/STT/sloveni/tokenizer/tokenizer_config.json',
 '/home/ec2-user/SageMaker/STT/sloveni/tokenizer/special_tokens_map.json',
 '/home/ec2-user/SageMaker/STT/sloveni/tokenizer/vocab.json',
 '/home/ec2-user/SageMaker/STT/sloveni/tokenizer/added_tokens.json')

In [72]:
!aws s3 cp  '/home/ec2-user/SageMaker/STT/sloveni/tokenizer'  s3://commonvoicesdataset/Sloveni/tokenizer --recursive

upload: sloveni/tokenizer/vocab.json to s3://commonvoicesdataset/Sloveni/tokenizer/vocab.json
upload: sloveni/tokenizer/special_tokens_map.json to s3://commonvoicesdataset/Sloveni/tokenizer/special_tokens_map.json
upload: sloveni/tokenizer/tokenizer_config.json to s3://commonvoicesdataset/Sloveni/tokenizer/tokenizer_config.json


In [21]:
# key = data['aws_access_key_id']
# secret = data['aws_secret_access_key']
# s3fsystems = s3fs.S3FileSystem(key=key, secret=secret)  

In [22]:
# tokenizer.save_pretrained(dataset_path)

('/home/ec2-user/SageMaker/STT/sloveni/tokenizer_config.json',
 '/home/ec2-user/SageMaker/STT/sloveni/special_tokens_map.json',
 '/home/ec2-user/SageMaker/STT/sloveni/vocab.json',
 '/home/ec2-user/SageMaker/STT/sloveni/added_tokens.json')

In [25]:
######

# save it to s3 in a folder for the language you want next to the json file 
#####



# tokenizer.save_pretrained('/home/ec2-user/SageMaker')



('/home/ec2-user/SageMaker/tokenizer_config.json',
 '/home/ec2-user/SageMaker/special_tokens_map.json',
 '/home/ec2-user/SageMaker/vocab.json',
 '/home/ec2-user/SageMaker/added_tokens.json')

In [73]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [74]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
%pwd

'/home/ec2-user/SageMaker/STT'

In [75]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test  = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [17]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [76]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [77]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

In [78]:
common_voice_train


Dataset({
    features: ['input_values', 'labels'],
    num_rows: 2594
})

In [25]:
aws_access_key_id=data['aws_access_key_id']
aws_secret_access_key=data['aws_secret_access_key']
s3  = boto3.resource('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)

In [ ]:

#from datasets.filesystems import S3FileSystem 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 KB 26.5 MB/s eta 0:00:00
  Created wheel for aiobotocore: filename=aiobotocore-2.1.1-py3-none-any.whl size=55483 sha256=bba75f387baccedfbd6e1f48c2e9a5a438ea3a2588aa7bbca3cfa2c78c46c080
  Stored in directory: /home/ec2-user/.cache/pip/wheels/52/fd/d3/e85dd95a98c12426ed6864dc24a408c3ca3253d4387b2f1cce
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.0
    Uninstalling botocore-1.24.0:
      Successfully uninstalled botocore-1.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [28]:
# key = data['aws_access_key_id']
# secret = data['aws_secret_access_key']
# s3fsystems = s3fs.S3FileSystem(key=key, secret=secret)  

In [79]:
common_voice_train.save_to_disk('s3://commonvoicesdataset/Sloveni/train_set',fs=s3fsystems)

In [80]:
common_voice_test.save_to_disk('s3://commonvoicesdataset/Sloveni/test_set',fs=s3fsystems)

In [32]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1647
})